In [1]:
%run ./ProlificAcademic_CrisisLogger_data_processing.ipynb #processed_transcriptions and specify_positive_data_by_survey variables
#hold the crisislogger and prolific academic data, respectively.
%run ./FocusGroup_data_processing.ipynb #all_focusgroup_processed_text holds the Focus group data.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans, Birch, AgglomerativeClustering, AffinityPropagation, DBSCAN, MiniBatchKMeans, MeanShift, SpectralClustering

c:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3209: DtypeWarning: Columns (93) have mixed types.Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
c:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3209: DtypeWarning: Columns (5,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [2]:
def collapse_list_of_strings(list_of_strings):
    return ' '.join(list_of_strings)

#processed_transcriptions_collapsed=[collapse_list_of_strings(answer) for answer in processed_transcriptions[0]]
processed_transcriptions_collapsed=[collapse_list_of_strings(answer) for answer in processed_transcriptions[0]]
count_vector=CountVectorizer()
DTM_data=count_vector.fit_transform(processed_transcriptions_collapsed)

In [3]:
def tf_construction(document):
    return document/sum(document)

def idf_construction(word_series):
    return word_series*math.log(word_series.shape[0]/sum(word_series))

def standardize_series(word_series):
    return(word_series-np.mean(word_series))/np.std(word_series)

inverted_vocabulary={y:x for x,y in count_vector.vocabulary_.items()}
DTM_data_df=pd.DataFrame(DTM_data.toarray())
DTM_data_df.rename(mapper=inverted_vocabulary, inplace=True, axis=1)
DTM_data_df=DTM_data_df.loc[:,DTM_data_df.apply(sum)>=5]
DTM_data_df=DTM_data_df.apply(tf_construction, axis=1)
DTM_data_df=DTM_data_df.apply(idf_construction, axis=0)
DTM_data_df=DTM_data_df.apply(standardize_series)
DTM_data_df.head()

,12,13,14,15,16,17,19,50,5th,ability,...,world,worried,worry,would,write,yeah,yet,york,young,zoom
0,-0.178366,-0.193402,-0.112924,-0.14487,-0.226861,-0.126785,-0.213956,-0.148501,-0.207382,-0.158311,...,-0.331785,-0.151232,-0.404283,-0.415306,-0.169268,-0.172152,-0.145208,-0.293064,-0.281939,-0.240268
1,-0.178366,-0.193402,-0.112924,-0.14487,-0.226861,-0.126785,-0.213956,-0.148501,2.519415,-0.158311,...,0.858918,-0.151232,-0.404283,0.575857,-0.169268,-0.172152,1.336663,-0.293064,0.928886,-0.240268
2,-0.178366,-0.193402,-0.112924,-0.14487,-0.226861,-0.126785,-0.213956,-0.148501,-0.207382,-0.158311,...,-0.331785,-0.151232,0.157804,-0.415306,-0.169268,-0.172152,-0.145208,0.898090,-0.281939,-0.240268
3,-0.178366,-0.193402,-0.112924,-0.14487,-0.226861,-0.126785,-0.213956,-0.148501,-0.207382,-0.158311,...,-0.331785,-0.151232,3.867576,2.122070,-0.169268,-0.172152,-0.145208,-0.293064,-0.281939,-0.240268
4,-0.178366,-0.193402,-0.112924,-0.14487,-0.226861,-0.126785,1.767927,-0.148501,-0.207382,-0.158311,...,6.441990,-0.151232,1.968972,-0.415306,-0.169268,-0.172152,-0.145208,-0.293064,-0.281939,-0.240268


In [4]:
number_of_clusters=5
clustering_model=KMeans(n_clusters=number_of_clusters, n_init=1000, init='random', max_iter=3000)
clustering_model.fit(DTM_data_df)
predicted_cluster=clustering_model.predict(DTM_data_df)
cluster_distribution=Counter(predicted_cluster)
cluster_distribution=np.array([value/DTM_data_df.shape[0] for key, value in cluster_distribution.items()])

In [5]:
print('Display distribution for KMeans cluster: \n')
print(np.round(cluster_distribution,2))

Display distribution for KMeans cluster: 

[0.96 0.01 0.01 0.01 0.01]


In [6]:
original_text=pd.DataFrame(processed_transcriptions[1])
original_text=pd.concat([original_text, pd.Series(predicted_cluster)],axis=1, join='inner')

cluster_centers_df=pd.DataFrame(clustering_model.cluster_centers_)
cluster_centers_rank_df=cluster_centers_df.rank(axis=1, method='first', ascending=True)
cluster_centers_rank_np=np.array(cluster_centers_rank_df)
cluster_centers_rank_np=cluster_centers_rank_np.astype(int)

In [7]:
for cluster in range(len(cluster_distribution)):
    words=DTM_data_df.columns.values
    word_ranks=pd.Series(cluster_centers_rank_np[cluster]-1)
    highest_tf_idf_scores=pd.Series(cluster_centers_df.iloc[cluster,:])
    
    word_rank_score_df=pd.DataFrame([words, word_ranks, highest_tf_idf_scores])
    word_rank_score_df.sort_values(by=1,axis=1, inplace=True)
    
    plt.barh(np.arange(10), list(word_rank_score_df.iloc[2,(word_rank_score_df.shape[1]-10):word_rank_score_df.shape[1]]))
    plt.yticks(np.arange(10), list(word_rank_score_df.iloc[0,(word_rank_score_df.shape[1]-10):word_rank_score_df.shape[1]]))
    plt.xlabel('Tf-Idf score')
    plt.title('Tf-Idf score for override theme #' + str(cluster+1) + ', \n % of overrides belonging to theme: ' + \
              str(np.round(cluster_distribution,2)[cluster]))
    plt.tight_layout()
    plt.savefig('Most_frequent_words_override_theme_' + str(cluster+1) + '.png', dpi=150)
    plt.close()